In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/My Drive/Kaggle/Chatbot')
!ls

 Chatbot.ipynb	'cornell movie-dialogs corpus'	 Cornell_movie_quotes_corpus


In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math

In [4]:
corpus_name = "cornell movie-dialogs corpus"
corpus = os.path.join("", corpus_name)

def print_line(file, n = 10):
  with open(file, 'rb') as datafile:
    lines = datafile.readlines()
  for line in lines[:n]:
    print(line)

print_line(os.path.join(corpus, "movie_lines.txt"))

b'L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!\n'
b'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!\n'
b'L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.\n'
b'L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?\n'
b"L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.\n"
b'L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow\n'
b"L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie.\n"
b'L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No\n'
b'L870 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?\n'
b'L869 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Like my fear of wearing pastels?\n'


In [5]:
def loadlines(fileName, fields):
  lines = {}
  with open(fileName, 'r', encoding='iso-8859-1') as f:
    for line in f:
      values = line.split("+++$+++")
      lineObj = {}

      for i, field in enumerate(fields):
        lineObj[field] = values[i]

      lines[lineObj['lineID']] = lineObj
  return lines

'cornell movie-dialogs corpus'

In [0]:
def loadConversations(fileName, lines, fields):
  conversations = []
  with open(fileName, 'r', encoding='iso-8859-1') as f:
    for line in f:
      values = line.split("+++$+++")

      convObj = {}
      for i, field in enumerate(fields):
        convObj[field] = values[i]
      
      utterrance_id_pattern = re.compile('L[0-9]+')
      lineIds = utterance_id_pattern.findall(convObj["utteranceIDs"])

      convObj["lines"] = []

      for lineId in lineIds:
        convObj["lines"].append(lines["lineId"])
      conversations.append(convObj)
  return conversations

In [0]:
def extractSentencePairs(conversations):
  qa_pairs = []
  for conversation in conversations:
    # Iterate over all lines of the conversations
    for i in range(len(conversation['lines']) - 1):
      inputLine = conversation["lines"][i]["text"].strip()
      targetLine = conversation["lines"][i+1]["text"].strip()

      if inputLine and targetLine:
        qa_pairs.append([inputLine, targetLine])
  return qa_pairs

In [0]:
datafile = os.path.join(corpus, "formatted_movie_lines.txt")

delimeter = '\t'

delimiter = str(codecs.decode(delimiter, "unicode_escape"))